In [1]:
import geopandas as gpd 
import pandas as pd 
import numpy as np 
from shapely.geometry import box 

In [2]:
import sys 
sys.path.append('/Users/gracecolverd/New_dataset')
from src import pre_process_building_data 

In [3]:
# build = gpd.read_file('/Volumes/T9/postcode_data/data/verisk_y2022_V5/bbox_139252.359375_526435.2583007812_149252.359375_536435.2583007812.gpkg')


In [4]:
# df = pre_process_building_data(build)

In [5]:
import concurrent.futures
import pandas as pd
import tempfile
import os
from src.fuel_calc import process_postcode_fuel
import shutil 
import tempfile
import threading
thread_local = threading.local()
from src import merge_temp_logs_to_main, generate_batch_list



def process_batch(pc_batch, data, gas_df, elec_df, temp_dir):

    log_file = os.path.join(temp_dir, 'log_file.csv' )
    # Collect results from processing each postcode into a list
    results = []
    for pc in pc_batch:
        
        pc_result = process_postcode_fuel(pc, data, gas_df, elec_df)
        results.append(pc_result)
    
    # Only proceed if we have results
    if results:
        df = pd.DataFrame(results)
        
        temp_file_path, is_first_write = get_thread_temp_file(log_file)
        # print('temp file path is ' , temp_file_path )
        # Open the file with 'a' mode to append and ensure headers are written correctly
        with open(temp_file_path, 'a') as f:
            df.to_csv(f, header=is_first_write, index=False)
        
        # Update the flag to indicate the header should not be written next time
        if is_first_write:
            thread_local.temp_file_first_write = False

        # print('temp file saved for batch')

def get_thread_temp_file(log_file):
    if not hasattr(thread_local, 'temp_file'):
        temp_dir = os.path.dirname(log_file)

        temp_file = tempfile.NamedTemporaryFile(delete=False, mode='w+', suffix='.csv', prefix='temp_log_', dir=temp_dir)        

        thread_local.temp_file = temp_file.name
        thread_local.temp_file_first_write = True
    return thread_local.temp_file, thread_local.temp_file_first_write


def merge_temp_files(temp_files, final_log_path):
    # Concatenate all temporary files into a single DataFrame and save it
    df_list = [pd.read_csv(temp_file) for temp_file in temp_files if temp_file]
    final_df = pd.concat(df_list, ignore_index=True)
    final_df.to_csv(final_log_path, index=False)
    
    # Optionally, remove temporary files after merging
    for temp_file in temp_files:
        os.remove(temp_file)

def main(pcs_list, data, gas_df, elec_df, temp_dir='/Users/gracecolverd/New_dataset/postcode_attrs/temp_files', max_workers=1, batch_size=5):
    # Ensure temporary directory exists
    os.makedirs(temp_dir, exist_ok=True)
    
    # Split pcs_list into batches for parallel processing
    pcs_batches = [pcs_list[i:i+batch_size] for i in range(0, len(pcs_list), batch_size)]
 
    

    # with concurrent.futures.ThreadPoolExecutor(max_workers = max_workers) as executor:
    #     futures = [executor.submit(process_batch, batch, data, gas_df, elec_df, temp_dir) for batch in pcs_batches]
    #     for future in concurrent.futures.as_completed(futures):
    #         temp_file_path = future.result()

    with concurrent.futures.ThreadPoolExecutor(max_workers = max_workers ) as executor:
        # Store futures if you need to wait for them or check for exceptions
        futures = []
        for i in range(0, len(pcs_list), batch_size):
            batch = pcs_list[i:i+batch_size]
    
            future= executor.submit(process_batch, batch, data, gas_df, elec_df, temp_dir )
            futures.append(future)
        concurrent.futures.wait(futures)

    
    # merge_temp_files(temp_files, final_log_path)



In [15]:
from src import find_postcode_for_ONSUD_file , find_data_pc


base_dir = '/Users/gracecolverd/New_dataset'
print('Finding data for ONSUD file')
pc_df , onsud_data = find_postcode_for_ONSUD_file('/Volumes/T9/Data_downloads/ONS_UPRN_database/ONSUD_DEC_2022/Data/ONSUD_DEC_2022_EE.csv' )
lab = 'EE'


pcs_list =  onsud_data.PCDS.unique().tolist()

INPUT_GPK= '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_22_updated_data/UKBuildings_Edition_15_new_format_upn.gpkg' 
fuel_year = 2022 
gas_elec_var= 'gas' 

print('Loading fuel df ', gas_elec_var)
gas_path =  f'/Volumes/T9/Data_downloads/UKGOV_Gas_elec/Postcode_level_gas_{fuel_year}.csv' #gas by postcode 
gas_df = pd.read_csv(gas_path) 

elec_path = f'/Volumes/T9/Data_downloads/UKGOV_Gas_elec/Postcode_level_all_meters_electricity_{fuel_year}.csv'
elec_df = pd.read_csv(elec_path) 

log = '/Users/gracecolverd/New_dataset/postcode_attrs/temp_files/log_file.csv' 

merge_temp_logs_to_main(log)

pc_list = generate_batch_list(pcs_list, log , 'postcode')

Finding data for ONSUD file
Len of missing rows  0
Loading fuel df  gas
Num of temp files found: 0
Starting cleanup
Start generate list
num to process 109100


In [8]:
# log = '/Users/gracecolverd/New_dataset/postcode_attrs/temp_files/log_file.csv' 
# pcs_list =  onsud_data.PCDS.unique().tolist()
# merge_temp_logs_to_main(log)

# pc_list = generate_batch_list(pcs_list, log , 'postcode')

In [16]:
len(pc_list), len(pcs_list)

(109100, 159695)

In [17]:

temp_dir = 'temp_files'
os.makedirs(temp_dir, exist_ok=True)    


print('Calc postcode vars')
main(pc_list, onsud_data, gas_df, elec_df, temp_dir,  max_workers=8, batch_size=50)


Calc postcode vars


/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key 

No data to process


/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key 

No data to process


...

/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key 

No data to process


/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key 

No data to process


/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key 

In [ ]:
fin_df



,postcode,res_total_buildings,mixed_total_buildings,comm_total_buildings,num_meters,total_gas,avg_gas,median_gas,all_types_total_buildings,all_types_build_vol_FGA_total,...,all_types_heated_vol_inc_basement_FGA_total,all_types_listed_bool_total,res_build_vol_FGA_total,res_base_floor_total,res_build_vol_inc_basement_FGA_total,res_heated_vol_EA_FGA_total,res_heated_vol_FGA_total,res_heated_vol_inc_basement_EA_FGA_total,res_heated_vol_inc_basement_FGA_total,res_listed_bool_total
0,MK17 9LH,-999,-999,-999,17,344929.860036,20289.991767,21414.212821,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EN7 6JU,48,0,0,49,551070.775694,11246.342361,10472.624613,48.0,22147.389867,...,14616.729961,0.0,22147.389867,0.0,22147.389867,14616.729961,14616.729961,14616.729961,14616.729961,0.0
2,EN7 6BQ,49,0,0,50,607825.686512,12156.513730,12222.567049,49.0,21430.280215,...,12879.999991,0.0,21430.280215,0.0,21430.280215,12879.999991,12879.999991,12879.999991,12879.999991,0.0
3,EN7 6LL,1,0,0,-999,-999.000000,-999.000000,-999.000000,1.0,3823.039923,...,2536.440042,0.0,3823.039923,0.0,3823.039923,2536.440042,2536.440042,2536.440042,2536.440042,0.0
4,EN7 6AP,13,0,0,15,111413.564700,7427.570980,6969.736407,13.0,6347.529922,...,3674.019951,0.0,6347.529922,0.0,6347.529922,3674.019951,3674.019951,3674.019951,3674.019951,0.0


In [ ]:

# dc = process_postcode(pc, data, fuel_df, 'gas')

In [ ]:
process_postcode(pc, data, fuel_df, 'gas')

Pre-processing building data...
Calculating volume metrics...
Producing clean building data...
Filtering non-commercial derelict premises...
DF passed tests
Pre process of building data complete


/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


{'postcode': 'EN7 6AP',
 'all_types_total_buildings': 13,
 'all_types_build_vol_FGA_total': 6347.5299219322205,
 'all_types_base_floor_total': 0,
 'all_types_build_vol_inc_basement_FGA_total': 6347.5299219322205,
 'all_types_heated_vol_EA_FGA_total': 3674.0199508666988,
 'all_types_heated_vol_FGA_total': 3674.0199508666988,
 'all_types_heated_vol_inc_basement_EA_FGA_total': 3674.0199508666988,
 'all_types_heated_vol_inc_basement_FGA_total': 3674.0199508666988,
 'all_types_listed_bool_total': 0,
 'res_total_buildings': 13,
 'res_build_vol_FGA_total': 6347.5299219322205,
 'res_base_floor_total': 0,
 'res_build_vol_inc_basement_FGA_total': 6347.5299219322205,
 'res_heated_vol_EA_FGA_total': 3674.0199508666988,
 'res_heated_vol_FGA_total': 3674.0199508666988,
 'res_heated_vol_inc_basement_EA_FGA_total': 3674.0199508666988,
 'res_heated_vol_inc_basement_FGA_total': 3674.0199508666988,
 'res_listed_bool_total': 0,
 'mixed_total_buildings': 0,
 'comm_total_buildings': 0,
 'num_meters': 15,
 '

In [ ]:
fuel[fuel['Postcode']==pc]['Num_meters'].values[0]  

NameError: name 'fuel' is not defined

In [ ]:
dc = calculate_postcode_attr(df)    

In [ ]:
dc

{'all_types_total_buildings': 48,
 'all_types_build_vol_FGA_total': 22147.389866838454,
 'all_types_base_floor_total': 0,
 'all_types_build_vol_inc_basement_FGA_total': 22147.389866838454,
 'all_types_heated_vol_EA_FGA_total': 14616.729961395262,
 'all_types_heated_vol_FGA_total': 14616.729961395262,
 'all_types_heated_vol_inc_basement_EA_FGA_total': 14616.729961395262,
 'all_types_heated_vol_inc_basement_FGA_total': 14616.729961395262,
 'all_types_listed_bool_total': 0,
 'res_total_buildings': 48,
 'res_build_vol_FGA_total': 22147.389866838454,
 'res_base_floor_total': 0,
 'res_build_vol_inc_basement_FGA_total': 22147.389866838454,
 'res_heated_vol_EA_FGA_total': 14616.729961395262,
 'res_heated_vol_FGA_total': 14616.729961395262,
 'res_heated_vol_inc_basement_EA_FGA_total': 14616.729961395262,
 'res_heated_vol_inc_basement_FGA_total': 14616.729961395262,
 'res_listed_bool_total': 0,
 'mixed_total_buildings': 0,
 'comm_total_buildings': 0}

In [ ]:
dc

{'all_types_total_buildings': 48,
 'all_types_build_vol_FGA_total': 22147.389866838454,
 'all_types_base_floor_total': 0,
 'all_types_build_vol_inc_basement_FGA_total': 22147.389866838454,
 'all_types_heated_vol_EA_FGA_total': 14616.729961395262,
 'all_types_heated_vol_FGA_total': 14616.729961395262,
 'all_types_heated_vol_inc_basement_EA_FGA_total': 14616.729961395262,
 'all_types_heated_vol_inc_basement_FGA_total': 14616.729961395262,
 'all_types_listed_bool_total': 0,
 'res_total_buildings': 48,
 'res_build_vol_FGA_total': 22147.389866838454,
 'res_base_floor_total': 0,
 'res_build_vol_inc_basement_FGA_total': 22147.389866838454,
 'res_heated_vol_EA_FGA_total': 14616.729961395262,
 'res_heated_vol_FGA_total': 14616.729961395262,
 'res_heated_vol_inc_basement_EA_FGA_total': 14616.729961395262,
 'res_heated_vol_inc_basement_FGA_total': 14616.729961395262,
 'res_listed_bool_total': 0}

In [ ]:
build['listed_grade'].value_counts()

Series([], Name: listed_grade, dtype: int64)

In [ ]:
def update_listed_type(df):
    df.loc[:, 'listed'] = df['listed_grade'].apply(lambda x: 1 if x is not None else 0)
    return df 

In [ ]:

build = update_listed_type(build)

In [ ]:
build

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,upn,geometry,listed
0,Unknown date,NaN,Residential,Domestic outbuilding,1,3.0,15.1,15.1,None,15.1,...,0,None,None,Unknown date,1,Residential,Residential,39974594,"MULTIPOLYGON (((141217.91870 526448.45920, 141...",0
1,Unknown date,NaN,Residential,Domestic outbuilding,1,3.0,16.1,16.1,None,16.1,...,0,None,None,Unknown date,1,Residential,Residential,39880696,"MULTIPOLYGON (((141266.19810 526452.21310, 141...",0
2,Unknown date,NaN,Residential,Domestic outbuilding,1,3.0,21.2,21.2,None,21.2,...,0,None,None,Unknown date,1,Residential,Residential,37816971,"MULTIPOLYGON (((141196.55049 526464.30430, 141...",0
3,1960-1979,NaN,Residential,Standard size semi detached,2,9.2,90.6,156.5,None,181.2,...,0,None,None,1960-1979,2,Residential,Residential,39928037,"MULTIPOLYGON (((141216.24700 526467.85010, 141...",0
4,Unknown date,NaN,Residential,Domestic outbuilding,1,3.0,21.2,21.2,None,21.2,...,0,None,None,Unknown date,1,Residential,Residential,39928017,"MULTIPOLYGON (((141200.29960 526466.85870, 141...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111710,1945-1959,NaN,Residential,Standard size semi detached,2,14.8,53.6,107.2,None,107.2,...,0,None,None,1945-1959,2,Residential,Residential,39901118,"MULTIPOLYGON (((143372.44860 532255.28810, 143...",0
111711,1945-1959,NaN,Residential,Standard size detached,2,6.7,62.8,62.8,None,125.6,...,0,None,None,1945-1959,2,Residential,Residential,39936066,"MULTIPOLYGON (((143336.14570 532263.05280, 143...",0
111712,1919-1944,NaN,Residential,Standard size detached,2,3.0,18.7,94.1,None,37.4,...,0,None,None,1919-1944,2,Residential,Residential,39923730,"MULTIPOLYGON (((143435.09040 532259.19290, 143...",0
111713,1945-1959,NaN,Residential,Standard size semi detached,2,7.8,54.6,109.2,None,109.2,...,0,None,None,1945-1959,2,Residential,Residential,39994340,"MULTIPOLYGON (((143352.50360 532248.82620, 143...",0
